In [1]:
% load_ext autoreload
% autoreload 2
% matplotlib inline

from __future__ import print_function
import random
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import scipy.io as sio
import os
import urllib.request as url

In [2]:
# if not os.path.exists('./dataset/train_32x32.mat'):
if not os.path.exists('./dataset'):
    os.mkdir('./dataset')
if not os.path.exists('./dataset/train_32x32.mat'):
    print('Start downloading train dataset...')
    url.urlretrieve("http://ufldl.stanford.edu/housenumbers/train_32x32.mat")
if not os.path.exists('./dataset/extra_32x32.mat'):
    print('Start downloading extra dataset...')
    url.urlretrieve("http://ufldl.stanford.edu/housenumbers/extra_32x32.mat")
if not os.path.exists('./dataset/test_32x32.mat'):
    print('Start downloading test dataset...')
    url.urlretrieve("http://ufldl.stanford.edu/housenumbers/test_32x32.mat")

train_set = sio.loadmat('./dataset/train_32x32.mat')
extra_set = sio.loadmat('./dataset/extra_32x32.mat')
test_set = sio.loadmat('./dataset/test_32x32.mat')

train_data = train_set['X']
train_labels = train_set['y']
extra_data = extra_set['X']
extra_labels = extra_set['y']
test_data = test_set['X']
test_labels = test_set['y']

# set the digit 0's lable to 0 rather than 10
train_labels[train_labels == 10] = 0
test_labels[test_labels == 10] = 0
extra_labels[extra_labels == 10] = 0


In [3]:
# The Google Cloud Platform cannot run this cell

# Build Validation Dataset and Labels Based on the Methods in This Paper:
# [https://arxiv.org/pdf/1204.3968.pdf]

num_labels = 10
valid_index = []
valid_index2 = []
train_index = []
train_index2 = []
for i in np.arange(num_labels):
    valid_index.extend(np.where(train_labels[:,0] == (i))[0][:400].tolist())
    train_index.extend(np.where(train_labels[:,0] == (i))[0][400:].tolist())
    valid_index2.extend(np.where(extra_labels[:,0] == (i))[0][:200].tolist())
    train_index2.extend(np.where(extra_labels[:,0] == (i))[0][200:].tolist())

random.shuffle(valid_index)
random.shuffle(train_index)
random.shuffle(valid_index2)
random.shuffle(train_index2)

valid_data = np.concatenate((extra_data[:,:,:,valid_index2], train_data[:,:,:,valid_index]), axis=3).transpose((3,0,1,2))
valid_labels = np.concatenate((extra_labels[valid_index2,:], train_labels[valid_index,:]), axis=0)[:,0]
train_data = np.concatenate((extra_data[:,:,:,train_index2], train_data[:,:,:,train_index]), axis=3).transpose((3,0,1,2))
train_labels = np.concatenate((extra_labels[train_index2,:], train_labels[train_index,:]), axis=0)[:,0]
test_data = test_data.transpose((3,0,1,2))
test_labels = test_labels[:,0]

print(train_data.shape, train_labels.shape)
print(test_data.shape, test_labels.shape)
print(valid_data.shape, valid_labels.shape)

(598388, 32, 32, 3) (598388,)
(26032, 32, 32, 3) (26032,)
(6000, 32, 32, 3) (6000,)


In [33]:
# no regularization
# This cell takes 6 hours
from binary import training
tf.reset_default_graph()
training(train_data, train_labels, valid_data, valid_labels, test_data, test_labels,
                     is_binary = False, 
                     is_stochastic = False, 
                     conv_featmap = [64, 64, 128, 128, 256, 256],
                     fc_units = [1024, 1024],
                     conv_kernel_size = [3, 3, 3, 3, 3, 3],
                     pooling_size = [2, 2, 2],
                     lr_start = 0.005,
                     lr_end = 0.001, 
                     epoch = 20,
                     batch_size = 50,
                     is_drop_out = False,
                     verbose = True,
                     pre_trained_model = None)

number of batches for training: 11967
epoch 1 
average train loss: 0.40063177357842034 ,  average accuracy : 93.94372881809127%
update learning rate:  [0.0046134042]
validation accuracy : 93.51666666666667%
* Best accuracy: 93.51666666666667%
epoch 2 
average train loss: 0.38907592865601315 ,  average accuracy : 96.73673268848974%
update learning rate:  [0.0042566997]
validation accuracy : 95.55%
* Best accuracy: 95.55%
epoch 3 
average train loss: 0.38643472897903836 ,  average accuracy : 97.4743811707454%
update learning rate:  [0.0039275754]
validation accuracy : 95.75%
* Best accuracy: 95.75%
epoch 4 
average train loss: 0.38491060232582214 ,  average accuracy : 97.8898306784227%
update learning rate:  [0.0036238986]
validation accuracy : 95.4%
epoch 5 
average train loss: 0.38382250934483125 ,  average accuracy : 98.20267117656103%
update learning rate:  [0.0033437018]
validation accuracy : 96.56666666666666%
* Best accuracy: 96.56666666666666%
epoch 6 
average train loss: 0.38312

In [34]:
# binary
# This cell takes 9 hours
from binary import training
tf.reset_default_graph()
training(train_data, train_labels, valid_data, valid_labels, test_data, test_labels,
                     is_binary = True, 
                     is_stochastic = False, 
                     conv_featmap = [64, 64, 128, 128, 256, 256],
                     fc_units = [1024, 1024],
                     conv_kernel_size = [3, 3, 3, 3, 3, 3],
                     pooling_size = [2, 2, 2],
                     lr_start = 0.01,
                     lr_end = 0.002, 
                     epoch = 30,
                     batch_size = 50,
                     is_drop_out = False,
                     verbose = True,
                     pre_trained_model = None)

number of batches for training: 11967
epoch 1 
average train loss: 0.40614821815078317 ,  average accuracy : 93.09695381591877%
update learning rate:  [0.0094776573]
validation accuracy : 91.63333333333333%
* Best accuracy: 91.63333333333333%
epoch 2 
average train loss: 0.39126257068074677 ,  average accuracy : 96.3284691537932%
update learning rate:  [0.0089825988]
validation accuracy : 93.83333333333333%
* Best accuracy: 93.83333333333333%
epoch 3 
average train loss: 0.38978316452286643 ,  average accuracy : 96.73656557283903%
update learning rate:  [0.0085133994]
validation accuracy : 94.81666666666666%
* Best accuracy: 94.81666666666666%
epoch 4 
average train loss: 0.3888383107027175 ,  average accuracy : 96.99325521233715%
update learning rate:  [0.0080687078]
validation accuracy : 95.5%
* Best accuracy: 95.5%
epoch 5 
average train loss: 0.3883832092047873 ,  average accuracy : 97.13881294410984%
update learning rate:  [0.0076472447]
validation accuracy : 95.25%
epoch 6 
avera

In [5]:
# stcochastic
# this cell takes 9 hours
from binary import training
tf.reset_default_graph()
training(train_data, train_labels, valid_data, valid_labels, test_data, test_labels,
                     is_binary = True, 
                     is_stochastic = True, 
                     conv_featmap = [64, 64, 128, 128, 256, 256],
                     fc_units = [1024, 1024],
                     conv_kernel_size = [3, 3, 3, 3, 3, 3],
                     pooling_size = [2, 2, 2],
                     lr_start = 0.01,
                     lr_end = 0.001, 
                     epoch = 30,
                     batch_size = 50,
                     is_drop_out = False,
                     verbose = True,
                     pre_trained_model = None)

number of batches for training: 11967
epoch 1 
average train loss: 0.5589829144137559 ,  average accuracy : 69.7168058182985%
update learning rate:  [0.0092611872]
validation accuracy : 82.3%
* Best accuracy: 82.3%
epoch 2 
average train loss: 0.40045780224131494 ,  average accuracy : 94.11134581575834%
update learning rate:  [0.0085769594]
validation accuracy : 81.6%
epoch 3 
average train loss: 0.397030263008545 ,  average accuracy : 94.96263294050014%
update learning rate:  [0.0079432828]
validation accuracy : 85.78333333333333%
* Best accuracy: 85.78333333333333%
epoch 4 
average train loss: 0.3954601864214279 ,  average accuracy : 95.36304203961309%
update learning rate:  [0.007356423]
validation accuracy : 87.0%
* Best accuracy: 87.0%
epoch 5 
average train loss: 0.39432288651389863 ,  average accuracy : 95.65683135356993%
update learning rate:  [0.0068129213]
validation accuracy : 81.3%
epoch 6 
average train loss: 0.39334532732830874 ,  average accuracy : 95.88511133244651%
upd